<a href="https://colab.research.google.com/github/cat-on-tree/HuggingFace-Tutorials/blob/main/5_BERT_%E6%83%85%E6%84%9F%E5%88%86%E7%B1%BB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 6.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "L

In [2]:
import torch
from datasets import load_dataset
from transformers import BertTokenizer
from transformers import BertModel
from torch.optim import AdamW

In [3]:
#检查GPU可用性，使用GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [4]:
#定义dataset方法
class Dataset(torch.utils.data.Dataset):
    def __init__(self, split):
        self.dataset = load_dataset(path='lansinuote/ChnSentiCorp', split=split)

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, i):
        text = self.dataset[i]['text']
        label = self.dataset[i]['label']

        return text, label

#获取训练集
dataset = Dataset('train')

len(dataset), dataset[0]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


dataset_infos.json:   0%|          | 0.00/960 [00:00<?, ?B/s]

(…)-00000-of-00001-02f200ca5f2a7868.parquet:   0%|          | 0.00/2.16M [00:00<?, ?B/s]

(…)-00000-of-00001-405befbaa3bcf1a2.parquet:   0%|          | 0.00/276k [00:00<?, ?B/s]

(…)-00000-of-00001-5372924f059fe767.parquet:   0%|          | 0.00/275k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9600 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1200 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1200 [00:00<?, ? examples/s]

(9600,
 ('选择珠江花园的原因就是方便，有电动扶梯直接到达海边，周围餐馆、食廊、商场、超市、摊位一应俱全。酒店装修一般，但还算整洁。 泳池在大堂的屋顶，因此很小，不过女儿倒是喜欢。 包的早餐是西式的，还算丰富。 服务吗，一般',
  1))

In [5]:
#加载字典和分词工具
token = BertTokenizer.from_pretrained('bert-base-chinese')

token

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

BertTokenizer(name_or_path='bert-base-chinese', vocab_size=21128, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [6]:
#dataset前处理
def collate_fn(data):
    sents = [i[0] for i in data]  #对语料库进行批量读取
    labels = [i[1] for i in data]

    #编码
    data = token.batch_encode_plus(batch_text_or_text_pairs=sents,
                                   truncation=True,
                                   padding='max_length',
                                   max_length=500,
                                   return_tensors='pt',
                                   return_length=True)

    #input_ids:编码之后的数字
    #attention_mask:是补零的位置是0,其他位置是1
    input_ids = data['input_ids']
    attention_mask = data['attention_mask']
    token_type_ids = data['token_type_ids']
    labels = torch.LongTensor(labels)
    # 将数据转移到 GPU
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)
    token_type_ids = token_type_ids.to(device)
    labels = labels.to(device)
    #print(data['length'], data['length'].max())

    return input_ids, attention_mask, token_type_ids, labels

In [7]:
#dataloader处理
loader = torch.utils.data.DataLoader(dataset=dataset,
                                     batch_size=16,
                                     collate_fn=collate_fn,
                                     shuffle=True,
                                     drop_last=True)

for i, (input_ids, attention_mask, token_type_ids,
        labels) in enumerate(loader):
    break

print(len(loader))
input_ids.shape, attention_mask.shape, token_type_ids.shape, labels

600


(torch.Size([16, 500]),
 torch.Size([16, 500]),
 torch.Size([16, 500]),
 tensor([1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0], device='cuda:0'))

In [8]:
#加载预训练模型
pretrained = BertModel.from_pretrained('bert-base-chinese')
pretrained = pretrained.to(device)
#不对预训练模型训练，无需计算梯度
for param in pretrained.parameters():
    param.requires_grad_(False)

#模型试算
check = pretrained(input_ids=input_ids,
           attention_mask=attention_mask,
           token_type_ids=token_type_ids)

check.last_hidden_state.shape

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

torch.Size([16, 500, 768])

In [9]:
#定义下游任务模型
class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = torch.nn.Linear(768, 2)

#前向传播
    def forward(self, input_ids, attention_mask, token_type_ids):
        with torch.no_grad():
            out = pretrained(input_ids=input_ids,
                       attention_mask=attention_mask,
                       token_type_ids=token_type_ids)

        out = self.fc(out.last_hidden_state[:, 0])
        #不使用自定义model层的写法如下：
        #out = torch.nn.Linear(768, 2)(out.last_hidden_state[:, 0])
        out = out.softmax(dim=1)

        return out


# 实例化模型并移动到 GPU
model = Model().to(device)

model(input_ids=input_ids,
      attention_mask=attention_mask,
      token_type_ids=token_type_ids).shape

torch.Size([16, 2])

In [10]:
# 定义损失函数和优化器
criterion = torch.nn.CrossEntropyLoss()  # 损失函数
optimizer = AdamW(model.parameters(), lr=5e-4)  # 优化器

# 定义训练参数
num_epochs = 3  # 总训练轮数
loss_threshold = 1e-4  # 损失变化阈值（早停条件）
prev_loss = float('inf')  # 初始化上一个 epoch 的 loss

for epoch in range(num_epochs):  # 外层循环负责 epoch
    model.train()  # 确保模型处于训练模式
    print(f"Epoch {epoch + 1}/{num_epochs}")

    epoch_loss = 0
    total_samples = 0  # 用于精确计算平均 loss

    for i, (input_ids, attention_mask, token_type_ids, labels) in enumerate(loader):
        # 前向传播
        out = model(input_ids=input_ids,
                    attention_mask=attention_mask,
                    token_type_ids=token_type_ids)

        # 计算损失
        loss = criterion(out, labels)

        # 反向传播
        optimizer.zero_grad()  # 清空梯度
        loss.backward()  # 反向传播
        optimizer.step()  # 更新参数

        # 累加 epoch 的 loss
        batch_size = input_ids.size(0)
        epoch_loss += loss.item() * batch_size  # 按样本权重累加
        total_samples += batch_size

        # 打印日志信息（每隔 50 个 batch 打印一次）
        if i % 50 == 0:
            out = out.argmax(dim=1)
            accuracy = (out == labels).sum().item() / len(labels)
            print(f"Batch {i}, Loss: {loss.item():.4f}, Accuracy: {accuracy:.4f}")

    # 计算 epoch 的平均 loss
    epoch_loss /= total_samples
    print(f"Epoch {epoch + 1}, Average Loss: {epoch_loss:.4f}")

    # 检查 Loss 是否变化不明显（早停机制）
    if abs(prev_loss - epoch_loss) < loss_threshold:
        print("Training loss converged. Stopping early.")
        break
    prev_loss = epoch_loss

Epoch 1/3
Batch 0, Loss: 0.7222, Accuracy: 0.4375
Batch 50, Loss: 0.5134, Accuracy: 1.0000
Batch 100, Loss: 0.4518, Accuracy: 0.9375
Batch 150, Loss: 0.5408, Accuracy: 0.8750
Batch 200, Loss: 0.4671, Accuracy: 0.8125
Batch 250, Loss: 0.4805, Accuracy: 0.8750
Batch 300, Loss: 0.5001, Accuracy: 0.8750
Batch 350, Loss: 0.4302, Accuracy: 0.8125
Batch 400, Loss: 0.4316, Accuracy: 0.9375
Batch 450, Loss: 0.4254, Accuracy: 0.8750
Batch 500, Loss: 0.4737, Accuracy: 0.8750
Batch 550, Loss: 0.5841, Accuracy: 0.6875
Epoch 1, Average Loss: 0.4696
Epoch 2/3
Batch 0, Loss: 0.4228, Accuracy: 0.9375
Batch 50, Loss: 0.5301, Accuracy: 0.7500
Batch 100, Loss: 0.3746, Accuracy: 0.9375
Batch 150, Loss: 0.3405, Accuracy: 1.0000
Batch 200, Loss: 0.4200, Accuracy: 0.8750
Batch 250, Loss: 0.4661, Accuracy: 0.8125
Batch 300, Loss: 0.4495, Accuracy: 0.8125
Batch 350, Loss: 0.4765, Accuracy: 0.8125
Batch 400, Loss: 0.4385, Accuracy: 0.8750
Batch 450, Loss: 0.4780, Accuracy: 0.8125
Batch 500, Loss: 0.4684, Accurac

In [12]:
def test():
    model.eval()  # 切换到评估模式
    correct = 0
    total = 0

    # 定义测试数据加载器
    loader_test = torch.utils.data.DataLoader(dataset=Dataset('validation'),
                                              batch_size=32,
                                              collate_fn=collate_fn,
                                              shuffle=True,
                                              drop_last=True)

    for i, (input_ids, attention_mask, token_type_ids, labels) in enumerate(loader_test):
        if i == 5:  # 测试只运行 5 个 batch
            break

        print(i)

        # 将数据转移到 GPU
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)
        labels = labels.to(device)

        # 前向传播
        with torch.no_grad():  # 禁用梯度计算
            out = model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        token_type_ids=token_type_ids)

        # 计算预测结果
        out = out.argmax(dim=1)
        correct += (out == labels).sum().item()
        total += len(labels)

    # 输出测试集的准确率
    print(correct / total)


test()

0
1
2
3
4
0.86875
